In [2]:
import nltk
nltk.download('punkt')

import pandas as pd
from nltk.tokenize import word_tokenize
from csv import QUOTE_NONE

# Read the SST data
sst_data = pd.read_csv("SST-2/train.tsv", sep="\t")

#Take 100 rows in validation set
validation_set = sst_data[:100]

#Take 100 rows in test set
test_set = sst_data[100:200]

#put remaining rows in training set
training_set = sst_data[200:]

# initialize the positive and negative count
positive_count = (training_set['label'] == 1).sum()
negative_count = (training_set['label'] == 0).sum()

#length of training set
training_length = len(training_set)

#getting prior probability for positive and negative classes
prior_probab_positive = positive_count / training_length
prior_probab_negative = negative_count / training_length

#printing prior probability of both classes
print(f"The prior probability of positive class is {prior_probab_positive}")
print(f"The prior probability of negative class is {prior_probab_negative}")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\soham\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


The prior probability of positive class is 0.5579681007907787
The prior probability of negative class is 0.44203189920922126


In [3]:
# A function to tokenized sentence and padding with start and end symbols
def sentence_tokenizer(sentence):
    tokenize_sentence = sentence.split()
    tokenize_sentence.insert(0, '<s>')
    tokenize_sentence.append('</s>')

    return tokenize_sentence

#tokenized the entire training dataset
tokenized_training_set = training_set['sentence'].apply(sentence_tokenizer)

#getting the first line from tokenized training set and printing it
first_line_trainingset = tokenized_training_set.iloc[0]
print(f"The tokenized first sentence is: {first_line_trainingset}")

#Getting a list of tokenized sentences in the training set
tokenizedlist_trainingset = tokenized_training_set.tolist()

total_tokens = []

#getting the unique words in total_tokens list
for i in tokenizedlist_trainingset:
    for j in i:
        total_tokens.append(j)

vocabulary_set = set(total_tokens)


vocabulary_size = len(vocabulary_set)                                       #getting the count of the unique words

print(f"The vocabulary size of training set is {vocabulary_size}")          #printing the count of unique words

The tokenized first sentence is: ['<s>', 'told', 'in', 'scattered', 'fashion', '</s>']
The vocabulary size of training set is 14813


In [4]:
# Function to count the biagram frequencies
def count_biagram_frequencies(tokenized_sequences):
    bigram_counts = {}                                                                  #Initializing a dictionary

    for tokens in tokenized_sequences:
        for i in range(len(tokens)-1):  
            first_word = tokens[i]                                                       # getting the first word
            second_word = tokens[i+1]                                                    # getting the second word

            bigram = (first_word, second_word)                                              
            if bigram not in bigram_counts:                                              # Getting the bigram counts
                bigram_counts[bigram]=0
            bigram_counts[bigram]+=1
    return bigram_counts


# Setting the bigram counts value in bigram_counts variable
bigram_counts = count_biagram_frequencies(tokenizedlist_trainingset)

#printing frequency of bigram '<s>' and 'the'
print(f"Frequency of bigram ('<s>', 'the') = {bigram_counts.get(('<s>', 'the'))}")

Frequency of bigram ('<s>', 'the') = 4450


In [5]:
import math

def smoothed_biagram_probab(wm, wm_1, bigram_counts, alpha, vocabulary_size):

    # count of biagram(wm, wm-1)
    count_bigram = bigram_counts.get((wm_1, wm), 0)

    #count of unigram(wm-1)
    count_unigram = 0
    for i, j in bigram_counts.items():
        if i[0] == wm_1:
            count_unigram +=j

    # To get smoothed probability using alpha smoothing formula
    smoothed_probability = (count_bigram + alpha) / (count_unigram + alpha * vocabulary_size)

    # TO get the negative log probability
    calc_negative_log_probab = math.log(smoothed_probability)

    return calc_negative_log_probab


word = 'award'
previous_word = 'academy'
# passing the parameters with alpha as 0.001 to the smoothed function to get the negative log probability
negative_log_probab1 = smoothed_biagram_probab(word, previous_word, bigram_counts, 0.001, 14813)

# passing the parameters with alpha as 0.5 to the smoothed function to get the negative log probability
negative_log_probab2 = smoothed_biagram_probab(word, previous_word, bigram_counts, 0.5, 14813)

# printing the negative log probability with alpha 0.001
print(f"The log probability with alpha= 0.001 is  {negative_log_probab1}")

# printing the negative log probability with alpha 0.5
print(f"The log probability with alpha= 0.5 is  {negative_log_probab2}")



The log probability with alpha= 0.001 is  -1.0250904304166832
The log probability with alpha= 0.5 is  -6.172912066128204


In [6]:
# Function for getting the log probability of a sentence
def sentence_log_probab(sentence, bigram_counts, alpha, vocabulary_size):
    # Splitting the sentence into words
    words_of_sentence = sentence.split()

    #Initializing the log probability    
    log_probability = 0.0

    # Looping through the words in sentence
    for i in range(1, len(words_of_sentence)):
        #Calculating negative log probability
        negative_log_probab = smoothed_biagram_probab(words_of_sentence[i], words_of_sentence[i-1], bigram_counts, alpha, vocabulary_size)
        log_probability+= negative_log_probab

    return log_probability

# Initializing first sentence, getting its log probability and printing it.
first_sentence = "this was a really great movie but it was a little too long."
log_probab1 = sentence_log_probab(first_sentence, bigram_counts, 0.1, 14813)
print(f"The log probability of first sentence is {log_probab1}")

# Initializing second sentence, getting its log probability and printing it.
second_sentence = "long too little a was it but movie great really a was this."
log_probab2 = sentence_log_probab(second_sentence, bigram_counts, 0.1, 14813)
print(f"The log probability of second sentence is {log_probab2}")

The log probability of first sentence is -69.91754047795261
The log probability of second sentence is -113.38139766388028


In [7]:
# All the alpha values
all_alpha_values = [0.001, 0.01, 0.1]

# initializing best log chances variable
best_log_chances = 0

# setting selected alpha as 0 initially
selected_alpha = 0

# looping through all the alpha values
for alpha in all_alpha_values:
    approximate_log_likelihood = 0

    # Applying log probability function to each sentence in the validation set
    for i in validation_set['sentence']:
        approximate_log_likelihood += sentence_log_probab(i, bigram_counts, alpha, vocabulary_size)

    # Printing the approximate log likelihood for current alpha
    print(f"The log likelihood estimate for alpha as {alpha} is {approximate_log_likelihood}")

    # Check which alpha gives the highest result
    if best_log_chances == 0 or approximate_log_likelihood > best_log_chances:
        best_log_chances = approximate_log_likelihood
        selected_alpha = alpha

# Printing the best alpha
print(f"\nThe best alpha is {selected_alpha}")


The log likelihood estimate for alpha as 0.001 is -3493.4581923786996
The log likelihood estimate for alpha as 0.01 is -3924.884625422819
The log likelihood estimate for alpha as 0.1 is -4795.08551812413

The best alpha is 0.001


In [8]:
# Seprating our positive and negative datasets
training_set_pos = training_set[training_set['label'] == 1]
training_set_neg = training_set[training_set['label'] == 0]

#Calculating vocabulary size for positive and negative sets
positive_vocabulary_size = len(set(''.join(training_set_pos['sentence']).split()))
negative_vocabulary_size = len(set(''.join(training_set_neg['sentence']).split()))

# Sentence tokenizer function
def sentence_tokenizer2(sentence):
    tokenize_sentence1 = sentence.split()
    return tokenize_sentence1

# Getting positive and negative bigram counts
positive_bigram_counts = count_biagram_frequencies(training_set_pos['sentence'].apply(sentence_tokenizer2).tolist())
negative_bigram_counts = count_biagram_frequencies(training_set_neg['sentence'].apply(sentence_tokenizer2).tolist())

print(len(positive_bigram_counts))
print(len(negative_bigram_counts))
# setting positive and negative score as a list
total_positive_score = []
total_negative_score = []

# looping through test set to get the positive and negative score
for sentence in test_set['sentence']:
    test_set_tokens = sentence_tokenizer2(sentence)

    positive_score = math.log(prior_probab_positive)
    negative_score = math.log(prior_probab_negative)

    # Applying the smoothed bigram probab function to get the positive and negative score
    for i in range(1, len(test_set_tokens)):
        word = test_set_tokens[i]
        previous_word = test_set_tokens[i-1]

        positive_score += smoothed_biagram_probab(word, previous_word, positive_bigram_counts, selected_alpha, positive_vocabulary_size)
        negative_score += smoothed_biagram_probab(word, previous_word, negative_bigram_counts, selected_alpha, negative_vocabulary_size)

    total_positive_score.append(positive_score)
    total_negative_score.append(negative_score)

predicted_labels = []

# Appending 1 as positive and 0 as negative in predicted labels
for i in range(len(test_set)):
    if total_positive_score[i] > total_negative_score[i]:
        predicted_labels.append(1)
    else:
        predicted_labels.append(0)

# Getting the original labels from test set
true_labels = test_set['label'].tolist()

# Setting positive predicted and negative predicted
positive_predicted =  predicted_labels.count(1)
negative_predicted =  predicted_labels.count(0) 

print(total_positive_score)
print(total_negative_score)
# initializing accurate predictions as a list
accurate_predictions = []

# Appending 1 for correct prediction and 0 for incorrect prediction
for true, predicted in zip(true_labels, predicted_labels):
    if true == predicted:
        accurate_predictions.append(1)
    else:
        accurate_predictions.append(0)

# Calculating the Accuracy
accuracy = sum(accurate_predictions) / len(predicted_labels) * 100

# printing predicted positive and predicted negative
print(f"Class Distribution - Predicted Positive: {positive_predicted} | Predicted Negative: {negative_predicted}")

# Printing the Accuracy
print(f"Predicted Accuracy : {accuracy}")

48522
44518
[-7.253391238232215, -9.519098921068263, -29.500317356639187, -43.00077704367012, -85.28020375046799, -30.84617820330108, -282.7169887981055, -26.085111406125037, -101.56336397163857, -53.83487418285453, -45.634460865658795, -3.3870862838919096, -95.6515466858324, -9.601853191031529, -73.49957423406723, -9.935380221314594, -192.5321198303463, -0.5834534852765388, -33.70501625313465, -173.02094432022972, -20.221760679446504, -23.185443911875076, -56.043350349467836, -110.30155724876961, -6.707926328068044, -107.99565989530217, -51.99499545528104, -38.163413478073316, -99.59810925611275, -17.10886455913073, -5.859496960799376, -88.8079853881296, -59.32297414072547, -9.935380221314594, -0.5834534852765388, -64.2603825041634, -71.75029148389306, -11.020180820821025, -22.095669013648184, -91.34455391009922, -21.54453748649091, -37.401668495187025, -125.38157160366583, -23.44038735914226, -32.4307466688923, -29.87100312373693, -82.79217564513755, -2.7887710491694855, -148.5139799